# Treinamento de Modelo para Detecção de Fake News

Este notebook realiza o treinamento dos modelos de machine learning para detecção de fake news e exporta os modelos treinados para o sistema de armazenamento S3.

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from IPython.display import display

# Configurações de visualização
plt.rcParams['figure.figsize'] = (10, 6)
sns.set_style("whitegrid")

print("Notebook de Treinamento - Random Forest")
print("="*50)

## 1. Carregamento dos Dados de Treino

Vamos carregar os dados que foram pré-processados no notebook 01.

In [ ]:
# Carregar dados de treino e teste preparados no notebook 01
print("Carregando dados de treino...")
print("="*40)

# Caminhos dos arquivos
data_dir = "../data/processed"
X_train_path = f"{data_dir}/X_train_fakebr.csv"
X_test_path = f"{data_dir}/X_test_fakebr.csv" 
y_train_path = f"{data_dir}/y_train_fakebr.csv"
y_test_path = f"{data_dir}/y_test_fakebr.csv"

# Verificar se os arquivos existem
required_files = [X_train_path, X_test_path, y_train_path, y_test_path]
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print(f"Arquivos faltando: {missing_files}")
    print("Execute primeiro o notebook 01_preprocess_fakebr.ipynb")
else:
    # Carregar os dados
    X_train = pd.read_csv(X_train_path)
    X_test = pd.read_csv(X_test_path)
    y_train = pd.read_csv(y_train_path)['REAL']  # Extrair a coluna REAL
    y_test = pd.read_csv(y_test_path)['REAL']
    
    print("Dados carregados com sucesso!")
    print(f"Treino: {X_train.shape[0]} amostras x {X_train.shape[1]} features")
    print(f"Teste: {X_test.shape[0]} amostras x {X_test.shape[1]} features")
    print(f"Distribuição do treino: {y_train.value_counts().to_dict()}")
    print(f"Distribuição do teste: {y_test.value_counts().to_dict()}")

    # Mostrar algumas features
    print(f"\nPrimeiras features: {list(X_train.columns[:5])}...")
    print(f"Total de features numéricas: {X_train.shape[1]}")

## 2. Treinamento do Random Forest (Baseline)

Vamos começar com um Random Forest simples, com parâmetros padrão, para estabelecer uma baseline.

In [ ]:
# Criar o modelo Random Forest com parâmetros padrão
print("Criando Random Forest baseline...")
print("="*40)

# Inicializar o modelo
rf_model = RandomForestClassifier(
    random_state=42,  # Para reprodutibilidade
    n_jobs=-1        # Usar todos os cores disponíveis
)

print("Modelo criado!")
print(f"Parâmetros padrão:")
print(f"   - n_estimators: {rf_model.n_estimators} árvores")
print(f"   - max_depth: {rf_model.max_depth} (sem limite)")
print(f"   - min_samples_split: {rf_model.min_samples_split}")
print(f"   - min_samples_leaf: {rf_model.min_samples_leaf}")

# Treinar o modelo
print(f"\nTreinando o modelo...")
rf_model.fit(X_train, y_train)
print("Treinamento concluído!")